# Creating a model to predict weather the customer is fraud or not

*Data - Customersdata.json

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
data  = pd.read_json('customersdata.json', lines=True)

## unpacking the Customers, orders, transactions and populating them into multiple columns

In [3]:
# Separating customers, orders, transactions

In [4]:
# Unpacking customer data
def unpack(df, column, fillna=None):
    ret = None
    if fillna is None:
        ret = pd.concat([df, pd.DataFrame((d for idx, d in df[column].iteritems()))], axis=1)
        del ret[column]
    else:
        ret = pd.concat([df, pd.DataFrame((d for idx, d in df[column].iteritems())).fillna(fillna)], axis=1)
        del ret[column]
    return ret

In [5]:
data.customer[0]

{'customerEmail': 'josephhoward@yahoo.com',
 'customerPhone': '400-108-5415',
 'customerDevice': 'yyeiaxpltf82440jnb3v',
 'customerIPAddress': '8.129.104.40',
 'customerBillingAddress': '5493 Jones Islands\nBrownside, CA 51896'}

In [6]:
type(data.customer[0])

dict

In [7]:
customer_data = unpack(data, 'customer', 0)

In [8]:
customer_data.head(3)

,fraudulent,orders,paymentMethods,transactions,customerBillingAddress,customerDevice,customerEmail,customerIPAddress,customerPhone
0,False,"[{'orderId': 'vjbdvd', 'orderAmount': 18, 'ord...","[{'paymentMethodId': 'wt07xm68b', 'paymentMeth...","[{'transactionId': 'a9lcj51r', 'orderId': 'vjb...","5493 Jones Islands\nBrownside, CA 51896",yyeiaxpltf82440jnb3v,josephhoward@yahoo.com,8.129.104.40,400-108-5415
1,True,"[{'orderId': 'nlghpa', 'orderAmount': 45, 'ord...","[{'paymentMethodId': 'y3xp697jx', 'paymentMeth...","[{'transactionId': '5mi94sfw', 'orderId': 'nlg...",356 Elizabeth Inlet Suite 120\nPort Joshuabury...,r0jpm7xaeqqa3kr6mzum,evansjeffery@yahoo.com,219.173.211.202,1-788-091-7546
2,False,"[{'orderId': 'yk34y2', 'orderAmount': 33, 'ord...","[{'paymentMethodId': '8pneoi03z', 'paymentMeth...","[{'transactionId': 'q3lyvbza', 'orderId': 'yk3...","8478 Sean Ridges Apt. 441\nDavisberg, PR 72250",4m7h5ipl1shyavt6vv2r,andersonwilliam@yahoo.com,67b7:3db8:67e0:3bea:b9d0:90c1:2b60:b9f0,024.420.0375


In [9]:
orders = customer_data[["customerEmail", "orders"]]

In [10]:
methods = customer_data[["customerEmail","paymentMethods"]]

In [11]:
transactions = customer_data[["customerEmail", "transactions"]]

In [12]:
tmp = orders.columns.tolist()

In [13]:
rows = []

In [14]:
orders_func = orders.apply(lambda row: [rows.append([row['customerEmail'], nn]) 
                         for nn in row.orders], axis=1)

In [15]:
orders = pd.DataFrame(rows, columns=tmp).set_index(['customerEmail'])

In [16]:
orders = orders.reset_index()

In [17]:
orders_data = unpack(orders, "orders", 0)

In [19]:
order_data = orders_data.drop_duplicates(subset="orderId")

In [20]:
rows = []

In [21]:
methods_func = methods.apply(lambda row: [rows.append([row['customerEmail'], nn]) 
                         for nn in row.paymentMethods], axis=1)

In [22]:
methods = pd.DataFrame(rows, columns=methods.columns).set_index(['customerEmail'])

In [23]:
methods.reset_index(inplace=True)

In [24]:
paymentMethods_data = unpack(methods, "paymentMethods", 0)

In [26]:
paymentMethods_data = paymentMethods_data.drop_duplicates(subset="paymentMethodId")

In [27]:
rows = []

In [28]:
tran_func = transactions.apply(lambda row: [rows.append([row['customerEmail'], nn]) 
                         for nn in row.transactions], axis=1)

In [29]:
transactions = pd.DataFrame(rows, columns=transactions.columns).set_index(['customerEmail'])

In [30]:
transactions.reset_index(inplace=True)

In [31]:
transactions_data = unpack(transactions, "transactions", 0)

In [32]:
transactions_data = transactions_data.drop_duplicates(subset="transactionId")

In [33]:
customer_data.columns

Index(['fraudulent', 'orders', 'paymentMethods', 'transactions',
       'customerBillingAddress', 'customerDevice', 'customerEmail',
       'customerIPAddress', 'customerPhone'],
      dtype='object')

In [34]:
customer_data = customer_data[["fraudulent", "customerEmail", "customerPhone", "customerDevice", 'customerIPAddress' ,'customerBillingAddress']]

In [35]:
customer_data.to_csv("customer_data.csv",header=True, index=False)

In [36]:
orders_data.to_csv("orders_data.csv", header=True, index=False)

In [35]:
paymentMethods_data.to_csv('payments_data.csv', header = True, index=False)

In [36]:
transactions_data.to_csv("transaction.csv",header=True, index = False)